<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

This notebook aims to provide functions that produce anchor boxes as decribed in the paper.

In [1]:
import numpy as np

In [2]:
image_test = np.random.randn(800,800)
image_test_feature = np.random.randn(50,50)
ratio = [0.5, 1, 2]
anchor_scales = [8, 16, 32]
gt_box = np.array([[20, 30, 400, 500], [300, 400, 500, 600]])

In [3]:
def anchor_box(center, ratio, scale, shape_initial, shape_featured):
    sub_width = shape_initial[0]/shape_featured[0]
    sub_height = shape_initial[1]/shape_featured[1]
    anchor_width = sub_width*scale*np.sqrt(ratio)
    anchor_height = sub_height*scale/np.sqrt(ratio)
    
    ym = center[1] - anchor_height/2
    yp = center[1] + anchor_height/2
    xm = center[0] - anchor_width/2
    xp = center[0] + anchor_width/2
    
    anchor = np.array((ym,xm,yp,xp))
    return(anchor)

In [4]:
def list_centers(shape_initial, shape_featured):
    ratio_h = shape_initial[1]/shape_featured[1]
    ratio_w = shape_initial[0]/shape_featured[0]
    #intiail center is the center at the left top corner
    all_centers = [np.array((ratio_w/2, ratio_h/2),dtype=float) + np.array((ratio_w*i, ratio_h*j),dtype=float) for i in range(int(shape_featured[0])) for j in range(int(shape_featured[1]))]
    return(all_centers)

In [5]:
def anchor_boxes(list_ratios, list_scales, shape_initial, shape_featured):
    list_center = list_centers(shape_initial, shape_featured)
    all_anchors = [anchor_box(center, ratio, scale,shape_initial,shape_featured) for center in list_center for ratio in list_ratios
                   for scale in list_scales]
    return(all_anchors)

In [6]:
def check_anchor_inside(anchor_box, shape_initial):
    ym = anchor_box[0]
    yp = anchor_box[2]
    xm = anchor_box[1]
    xp = anchor_box[3]
    is_inside = (min(xm,xp)>0) & (max(xm,xp)<shape_initial[0]) & (max(yp,ym) < shape_initial[1]) & (min(ym,yp) > 0) 
    return(is_inside)

In [7]:
def iou(box1,box2):
    xm = max(box1[1], box2[1])
    xp = min(box1[3], box2[3])
    ym = max(box1[0], box2[0])
    yp = min(box1[2], box2[2])
    
    intersection = 0
    
    if((xm < xp) &(ym < yp)):
        intersection = (xp - xm)*(yp-ym)
    
    union = (box1[3]-box1[1])*(box1[2] - box1[0]) + (box2[3]-box2[1])*(box2[2] - box2[0]) - intersection
    return(intersection/union)

In [8]:
iou(np.array([1,1,3,3]),np.array([2,2,4,4]))

0.14285714285714285

In [9]:
list_anchors = anchor_boxes(ratio,anchor_scales,(800,800),(50,50))

In [10]:
def iou_anchors_vs_gtbox(list_anchors, list_gt_box):
    res = np.transpose([[iou(anchor, gt_box) for anchor in list_anchors] for gt_box in list_gt_box])
    return(np.array(res))

In [77]:
#Return an array with :
#for all ground truth box, the anchors which maximize the IOU with it
#for all anchor, the max of the IOU

#the first column of the array is the index and the last the IOU 
def best_anchors_from_iou(dt_anchors_vs_gtbox):
    #index highest by gtbox (cond a)
    dt_anchors_vs_gtbox.argmax(axis = 0)
    ind_argmax = np.where(dt_anchors_vs_gtbox == dt_anchors_vs_gtbox.max(axis = 0))[0]
    cond_a = dt_anchors_vs_gtbox[ind_argmax,:].max(axis = 1)
    
    #highest by anchors box (cond b)
    index = dt_anchors_vs_gtbox.argmax(axis = 1)
    iou_max = dt_anchors_vs_gtbox.max(axis = 1)
    cond_b = dt_anchors_vs_gtbox[np.arange(dt_anchors_vs_gtbox.shape[0]),index]
    
    index_res = np.concatenate((ind_argmax,np.arange(dt_anchors_vs_gtbox.shape[0])))
    res = np.concatenate((cond_a, cond_b), axis=0)
    res = np.column_stack((index_res,res))
    return(res)

In [84]:
#label_from_iou returns a np.array containing for each anchor its label. (+1 if foreground, 0 if background and -1 if not used
#during the learning phase)
#The default thresholds are defined according the original paper about Fatest RCNN.

def label_from_iou(dt_anchors_vs_gtbox,pos_threshold = 0.7, neg_threshold = 0.3):
    label = np.full(dt_anchors_vs_gtbox.shape[0],-1)
    iou_max = dt_anchors_vs_gtbox.max(axis = 1)
    #positive labels : 1
    label[iou_max > pos_threshold] = 1
    #negative labels : 0
    label[iou_max < neg_threshold] = 0
    #for anchors whose maximize IOU for a given object : +1
    dt_anchors_vs_gtbox.argmax(axis = 0)
    ind_argmax = np.where(dt_anchors_vs_gtbox == dt_anchors_vs_gtbox.max(axis = 0))[0]
    label[ind_argmax] = 1
    return(label)

In [103]:
lab = label_from_iou(iou_anchors_vs_gtbox(list_anchors, gt_box))
len((np.where((lab == 1) | (lab == 0)))[0])

19735

In [78]:
best_anchors_from_iou(iou_anchors_vs_gtbox(list_anchors, gt_box))

array([[6.83600000e+03, 6.81304932e-01],
       [6.84500000e+03, 6.81304932e-01],
       [6.85400000e+03, 6.81304932e-01],
       ...,
       [2.24970000e+04, 0.00000000e+00],
       [2.24980000e+04, 0.00000000e+00],
       [2.24990000e+04, 0.00000000e+00]])

In [116]:
#TODO : heck how to fill when

def batch_training_proposal(dt_anchors_vs_gtbox, nsize = 256, pos_ratio = 0.5):
    #number of positive units we need to reach in the training sample (we want a balanced sample)
    nb_pos_to_drawn = round(nsize*pos_ratio)
    lab = label_from_iou(dt_anchors_vs_gtbox)
    pos_lab = np.where(lab == 1)[0]
    neg_lab = np.where(lab == 0)[0]
    pos = len(pos_lab)
    neg = len(neg_lab)
    
    if (pos > nb_pos_to_drawn):
        disabled_index_pos = np.random.choice(pos_lab, size=(pos - nb_pos_to_drawn), replace = False)
        lab[disabled_index_pos] = -1
    
    if (neg > nsize - nb_pos_to_drawn):
        if(pos < nb_pos_to_drawn):
            disabled_index_neg = np.random.choice(neg_lab, size=(neg - 2*nsize + pos), replace = False)
        else:
            disabled_index_neg = np.random.choice(neg_lab, size=(neg + nb_pos_to_drawn - nsize), replace = False)
        
        lab[disabled_index_neg] = -1    
    
    res = np.where((lab == 0) | (lab == 1))[0]
    return res

In [119]:
(batch_training_proposal(iou_anchors_vs_gtbox(list_anchors, gt_box))).shape

(512,)